In [18]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

In [27]:
url="https://www.indiabix.com/aptitude/problems-on-trains/038003"
response = requests.get(url, verify=False)

d:\Web_Scraper\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [28]:
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    
    questions = []
    options_list = []
    answers = []
    topics = []

    # Extract the topic (e.g., Problems on Trains)
    topic_element = soup.find('div', class_='h5 w-100 mb-3 p-0')
    topic = topic_element.text.strip() if topic_element else "Unknown"

    # Find all question blocks
    for question_block in soup.find_all('div', class_='bix-div-container'):
        # Extract the question
        question_element = question_block.find('div', class_='bix-td-qtxt')
        if question_element:
            question_text = question_element.text.strip()
        else:
            question_text = "Question not found"  # Handle missing questions
        
        # Extract the options
        options = []
        for option_block in question_block.find_all('div', class_='bix-td-option-val'):
            option_text = option_block.text.strip()
            options.append(option_text)

        # Extract the correct answer (letter) and match it to the actual option text
        answer_element = question_block.find('input', class_='jq-hdnakq')
        if answer_element:
            correct_option_letter = answer_element['value']  # The answer will be A, B, C, or D
            # Match the letter to the corresponding option text
            option_mapping = {
                'A': options[0] if len(options) > 0 else '',
                'B': options[1] if len(options) > 1 else '',
                'C': options[2] if len(options) > 2 else '',
                'D': options[3] if len(options) > 3 else ''
            }
            correct_answer = option_mapping.get(correct_option_letter, "Answer not found")
        else:
            correct_answer = "Answer not found"  # Handle missing answers

        # Append data to lists
        topics.append(topic)
        questions.append(question_text)
        options_list.append(options)
        answers.append(correct_answer)


In [29]:
# Create a DataFrame
data = {
    'Topic':topic,
    'Question': questions,
    'Option 1': [options[0] if len(options) > 0 else '' for options in options_list],
    'Option 2': [options[1] if len(options) > 1 else '' for options in options_list],
    'Option 3': [options[2] if len(options) > 2 else '' for options in options_list],
    'Option 4': [options[3] if len(options) > 3 else '' for options in options_list],
    'Answer': answers
}


In [30]:
# Define the file path for the CSV
csv_file = 'aptitude_questions_with_answers.csv'

# Create a DataFrame from the scraped data
df = pd.DataFrame(data)

# Check if the file already exists
if os.path.isfile(csv_file):
    # Append data to the existing CSV without writing the header
    df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    # Write a new CSV with the header
    df.to_csv(csv_file, mode='w', header=True, index=False)

print("Data has been appended to aptitude_questions_with_answers.csv")

Data has been appended to aptitude_questions_with_answers.csv
